In [1]:
!pip install phe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 1.1 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from scipy.special import expit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from phe import paillier
import time
import psutil
import os
from threading import Thread, Lock

In [3]:
X_train = pd.read_csv('X_train_classification.csv')
X_train=X_train.to_numpy()
y_train = pd.read_csv('y_train_classification.csv')
y_train=y_train.to_numpy().reshape(-1)
X_test = pd.read_csv('X_test_classification.csv')
X_test=X_test.to_numpy()
y_test = pd.read_csv('y_test_classification.csv')
y_test=y_test.to_numpy().reshape(-1)

y_train = np.where(y_train == 0, -1, 1)
y_test = np.where(y_test == 0, -1, 1)

n_samples,n_attributes=X_train.shape
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
bias = 0
lr = 0.01
C = 0.01 # change C=10 or 1 if high attributes are involved

In [4]:
# Paillier key generation
public_key, private_key = paillier.generate_paillier_keypair()

In [5]:
# Lock for controlling CPU monitoring
monitor_lock = Lock()

In [6]:
# Function to handle encrypted multiplication at the edge device
def paillier_multiplication_edge(a, b):
    with monitor_lock:  # Acquire lock to exclude this part from CPU and RAM monitoring
        a = private_key.decrypt(a)
        a=round(a,6)
        a = a * b
    return a

In [7]:
# Function to handle encrypted multiplication at the cloud
def paillier_multiplication_cloud(a, b):
    noise1 = 0.01
    noise2 = 0.02
    a = a + noise1
    sub_mass = (b * noise1) * (-1)
    sol = paillier_multiplication_edge(a, b)
    sol = sol + sub_mass
    return sol

# Function to handle encrypted multiplication at the cloud
# def paillier_multiplication_cloud(a, b):
#     noise1 = 10
#     noise2 = 20
#     # Encrypt the noise before adding it to the encrypted number
#     encrypted_noise1 = public_key.encrypt(noise1)
#     a = a + encrypted_noise1  # Now you are adding two EncryptedNumber objects
#     sub_mass = (b * noise1) * (-1)
#     sol = paillier_multiplication_edge(a, b)
#     sol = sol + sub_mass
#     return sol

In [8]:
# enc_X_train = [[public_key.encrypt(j) for j in i] for i in X_train]
# enc_y_train = [public_key.encrypt(int(i)) for i in y_train]
enc_X_train=[]
count=0
for i in X_train:
  row_in=[]
  for j in i:
    row_in.append(public_key.encrypt(j))
  print(f"row {count} encrypted")
  enc_X_train.append(row_in)
  count+=1

enc_y_train = [public_key.encrypt(int(i)) for i in y_train]

row 0 encrypted
row 1 encrypted
row 2 encrypted
row 3 encrypted
row 4 encrypted
row 5 encrypted
row 6 encrypted
row 7 encrypted
row 8 encrypted
row 9 encrypted
row 10 encrypted
row 11 encrypted
row 12 encrypted
row 13 encrypted
row 14 encrypted
row 15 encrypted
row 16 encrypted
row 17 encrypted
row 18 encrypted
row 19 encrypted
row 20 encrypted
row 21 encrypted
row 22 encrypted
row 23 encrypted
row 24 encrypted
row 25 encrypted
row 26 encrypted
row 27 encrypted
row 28 encrypted
row 29 encrypted
row 30 encrypted
row 31 encrypted
row 32 encrypted
row 33 encrypted
row 34 encrypted
row 35 encrypted
row 36 encrypted
row 37 encrypted
row 38 encrypted
row 39 encrypted
row 40 encrypted
row 41 encrypted
row 42 encrypted
row 43 encrypted
row 44 encrypted
row 45 encrypted
row 46 encrypted
row 47 encrypted
row 48 encrypted
row 49 encrypted
row 50 encrypted
row 51 encrypted
row 52 encrypted
row 53 encrypted
row 54 encrypted
row 55 encrypted
row 56 encrypted
row 57 encrypted
row 58 encrypted
row 59 

In [9]:
len(enc_X_train)

80

In [10]:
def paillier_scalar_rescaling(r):
  r=private_key.decrypt(r)
  r=round(r,7)
  r=public_key.encrypt(r)
  return r

In [11]:
# margin calculation:
weights=[]
for i in range(X_train.shape[1]):
  weights.append(public_key.encrypt(0))

weights

In [12]:
for _ in range(3):
 for w in range(n_samples):
    xi=enc_X_train[w]
    yi=enc_y_train[w]

    summ=0
    for i,j in zip(weights,xi):
      r=None
      r=paillier_multiplication_cloud(j,i)
      mm=summ+r

    mm=paillier_scalar_rescaling(mm)
    l2=mm+bias


    lot=paillier_multiplication_cloud(yi,l2)
    # margins=1-summ

    if private_key.decrypt(lot) < 1:
      s=C*yi
      m2=[]
      for m in xi:
        m2.append(paillier_multiplication_cloud(m,s))

      r3=[]
      for i,j in zip(weights,m2):
        r3.append(i-j)

      r4=[]
      for i in r3:
        r4.append(lr*i)

      r5=[]
      for i,j in zip(weights,r4):
        r5.append(i-j)

      weights=r5
      bias=bias-lr*(-C*yi)

    else:
      r6=[]
      for i in weights:
        m=i*lr
        r6.append(i-m)

      weights=r6

    print(f"epoch = {_} iter {w} executed")


    if w%5==0:
      weights2=[]
      for i in weights:
        weights2.append(paillier_scalar_rescaling(i))
      weights=weights2
      bias=paillier_scalar_rescaling(bias)


epoch = 0 iter 0 executed
epoch = 0 iter 1 executed
epoch = 0 iter 2 executed
epoch = 0 iter 3 executed
epoch = 0 iter 4 executed
epoch = 0 iter 5 executed
epoch = 0 iter 6 executed
epoch = 0 iter 7 executed
epoch = 0 iter 8 executed
epoch = 0 iter 9 executed
epoch = 0 iter 10 executed
epoch = 0 iter 11 executed
epoch = 0 iter 12 executed
epoch = 0 iter 13 executed
epoch = 0 iter 14 executed
epoch = 0 iter 15 executed
epoch = 0 iter 16 executed
epoch = 0 iter 17 executed
epoch = 0 iter 18 executed
epoch = 0 iter 19 executed
epoch = 0 iter 20 executed
epoch = 0 iter 21 executed
epoch = 0 iter 22 executed
epoch = 0 iter 23 executed
epoch = 0 iter 24 executed
epoch = 0 iter 25 executed
epoch = 0 iter 26 executed
epoch = 0 iter 27 executed
epoch = 0 iter 28 executed
epoch = 0 iter 29 executed
epoch = 0 iter 30 executed
epoch = 0 iter 31 executed
epoch = 0 iter 32 executed
epoch = 0 iter 33 executed
epoch = 0 iter 34 executed
epoch = 0 iter 35 executed
epoch = 0 iter 36 executed
epoch = 0 i

In [13]:
for i in weights:
  print(private_key.decrypt(i))

0.00020532118008260002
0.0071579019786138


In [14]:
summ

0

In [15]:
private_key.decrypt(bias)

0.0

In [16]:
enc_X_test=[]
count=0
for i in X_test:
  row_in=[]
  for j in i:
    row_in.append(public_key.encrypt(j))
  print(f"row {count} encrypted")
  enc_X_test.append(row_in)
  count+=1

row 0 encrypted
row 1 encrypted
row 2 encrypted
row 3 encrypted
row 4 encrypted
row 5 encrypted
row 6 encrypted
row 7 encrypted
row 8 encrypted
row 9 encrypted
row 10 encrypted
row 11 encrypted
row 12 encrypted
row 13 encrypted
row 14 encrypted
row 15 encrypted
row 16 encrypted
row 17 encrypted
row 18 encrypted
row 19 encrypted


In [17]:
def return_sign(x):
  x1=private_key.decrypt(x)
  return np.sign(x1)

In [18]:
y_pred=[]
count2=0
for xi in enc_X_test:
  for i,j in zip(xi,weights):
    r=paillier_multiplication_cloud(i,j)
  p=r+bias
  y_pred.append(return_sign(p))
  print(f"item {count2} tested")

item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested
item 0 tested


In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.85